In [ ]:
from PyQt5.QtWidgets import (QMainWindow, QApplication, QPushButton, QWidget, QAction, 
                             QTabWidget,QVBoxLayout, QHBoxLayout, QInputDialog, QLineEdit, QLabel,
                             QFileDialog, QMainWindow, QPushButton, QTextEdit, QMessageBox)
from PyQt5.QtGui import QIcon, QTextCursor
from PyQt5.QtCore import pyqtSlot, QCoreApplication, QProcess, QObject, pyqtSignal
import sys
import os

In [20]:
class EmittingStream(QObject):

    textWritten = pyqtSignal(str)

    def write(self, text):

        self.textWritten.emit(str(text))
    
    def flush(self):
        pass

    
class App(QMainWindow):        
 
    def __init__(self):   
        super(App, self).__init__()
        
        self.title = 'Site Data Utilities'
        self.left = 0
        self.top = 0
        self.width = 600
        self.height = 400
        self.setWindowTitle(self.title)
        self.setWindowIcon(QIcon('/vol01/active_projects/anthony/brain.jpg'))
        self.setGeometry(self.left, self.top, self.width, self.height)
        
 
        # Initialize tab widget
        self.tabs = QTabWidget()
        
        # create tabs 
        self.h1Tab = QWidget()
        self.neuroTab = QWidget()
        self.redirectTab = QWidget()
        
        #################################### ERP TAB ####################################
        
        # create vertical layout for ERP tab 
        self.erpLayout = QVBoxLayout()
    
        # create directory box 
        self.dirLayout = QHBoxLayout()
        dirLabel = QLabel('Directory: ')
        self.dirInp = QLineEdit('/vol01/active_projects/anthony/ns650')
        self.dirLayout.addWidget(dirLabel)
        self.dirLayout.addWidget(self.dirInp)
        
        # create experiment box 
        self.filesLayout = QHBoxLayout()
        self.filesLabel = QLabel('Space-delimited exp names: ')
        self.filesInp = QLineEdit('aod ans')
        self.filesLayout.addWidget(self.filesLabel)
        self.filesLayout.addWidget(self.filesInp)
        

        # create target directory box 
        self.trgLayout = QHBoxLayout()
        self.trgLabel = QLabel('Target Directory: ')
        self.trgInp = QLineEdit('/vol01/active_projects/anthony/test_qt')
        self.trgLayout.addWidget(self.trgLabel)
        self.trgLayout.addWidget(self.trgInp)
        
         ############################################## TEXT EDITOR ##############################################
        
        # PROCESS IS A WIDGET NOT A LAYOUT 
        #self.process  = QTextEdit()
        #self.process.moveCursor(QTextCursor.Start)
        #self.process.ensureCursorVisible()
        #self.process.setLineWrapColumnOrWidth(1000)
        #self.process.setLineWrapMode(QTextEdit.FixedPixelWidth)
        
        ############################################## END EDITOR ##############################################
        
        #################################### ERP BUTTONS 
        # add create get h1's button
        self.h1Button = QPushButton("Create h1 files")
        self.h1Button.clicked.connect(self.h1_handler)
        self.h1Button.setToolTip("Instructions:\nINPUTS => Directory\nINPUTS => Experiments\nINPUTS => Target Directory")
        
        # add review data button 
        self.reviewButton = QPushButton("Review Data")
        self.reviewButton.clicked.connect(self.review_handler)
        self.reviewButton.setToolTip("Instructions:\nINPUTS => Directory")

        # add david's shell scripts 
        self.davidButton = QPushButton("Run ERP shell scripts")
        self.davidButton.clicked.connect(self.erp_shell_scripts)
        self.davidButton.setToolTip("Instructions:\nINPUTS => Directory")
        self.davidButton.resize(100,32)
        
        # add a clear button for editor
        #clearButton = QPushButton('Clear all text')
        #clearButton.clicked.connect(self.clear_function)
        

        # add dirLayout, files layout, start buttons to nav layout 
        self.erpLayout.addLayout(self.dirLayout)
        self.erpLayout.addLayout(self.filesLayout)
        self.erpLayout.addLayout(self.trgLayout)
        
        # add buttons 
        self.erpLayout.addWidget(self.h1Button)
        self.erpLayout.addWidget(self.reviewButton)
        self.erpLayout.addWidget(self.davidButton)
        
        # add nav layout to NAV TAB 
        self.h1Tab.setLayout(self.erpLayout)
        
        #################################### NEURO TAB ####################################
        
        # create vertical layout for neuropsych tab 
        self.neuroLayout = QVBoxLayout()
    
        # create directory box -- neuropsych 
        self.dirLayoutNeuro = QHBoxLayout()
        dirLabelNeuro = QLabel('Directory: ')
        self.dirInpNeuro = QLineEdit('/vol01/raw_data/staging/ucsd/oct_neuro')
        self.dirLayoutNeuro.addWidget(dirLabelNeuro)
        self.dirLayoutNeuro.addWidget(self.dirInpNeuro)
        
        
        # create month box -- neuropsych 
        self.monthLayoutNeuro = QHBoxLayout()
        monthLabelNeuro = QLabel('Enter month as integer: ')
        self.monthInpNeuro = QLineEdit('10')
        self.monthLayoutNeuro.addWidget(monthLabelNeuro)
        self.monthLayoutNeuro.addWidget(self.monthInpNeuro)
        
        # create site box -- neuropsych 
        self.siteLayoutNeuro = QHBoxLayout()
        siteLabelNeuro = QLabel('Enter site name: ')
        self.siteInpNeuro = QLineEdit('site here')
        self.siteLayoutNeuro.addWidget(siteLabelNeuro)
        self.siteLayoutNeuro.addWidget(self.siteInpNeuro)
        
        # add directory, month, & site box to neuroLayout    
        self.neuroLayout.addLayout(self.dirLayoutNeuro)
        self.neuroLayout.addLayout(self.monthLayoutNeuro)
        self.neuroLayout.addLayout(self.siteLayoutNeuro)
        
        # add neuroLayout to neuroTab
        self.neuroTab.setLayout(self.neuroLayout)
        
        #################################### NEURO BUTTONS 
        # add check neuro raw data button 
        self.checkButtonNeuro = QPushButton("Check /raw_data/neuropsych/__sitename__")
        self.checkButtonNeuro.clicked.connect(self.check_neuro_handler)
        self.checkButtonNeuro.setToolTip('INSTRUCTIONS:\nINPUTS => Dirctory\nINPUTS => Site Name')
        
        # add check for duplicates button 
        self.dupesButtonNeuro = QPushButton("Check for Duplicates")
        self.dupesButtonNeuro.clicked.connect(self.dupes_neuro_handler)
        self.dupesButtonNeuro.setToolTip('INSTRUCTIONS:\nINPUTS => Directory')
        
        # add neuro review button 
        self.reviewButtonNeuro = QPushButton("Run neuropsych check")
        self.reviewButtonNeuro.clicked.connect(self.review_neuro_handler)
        self.reviewButtonNeuro.setToolTip('INSTRUCTIONS:\nINPUTS => Directory\nINPUTS => Month')
    
        # add move neuro files button 
        self.moveButtonNeuro = QPushButton("MOVE files to /raw_data/neuropsych/__sitename__")
        self.moveButtonNeuro.clicked.connect(self.move_neuro_handler)
        
        # add buttons to neuroLayout
        self.neuroLayout.addWidget(self.checkButtonNeuro)
        self.neuroLayout.addWidget(self.dupesButtonNeuro)
        self.neuroLayout.addWidget(self.reviewButtonNeuro)
        self.neuroLayout.addWidget(self.moveButtonNeuro)
        
        #################################################### END NEURO TAB ####################################################

        #################################################### new std out tab 
        
        self.processLayout = QVBoxLayout()
        
        self.process  = QTextEdit()
        
        
        # add a clear button for editor
        clearButton = QPushButton('Clear all text')
        clearButton.clicked.connect(self.clear_function)
        
        self.processLayout.addWidget(self.process)
        self.processLayout.addWidget(clearButton)
        
        self.redirectTab.setLayout(self.processLayout)
        

        
        #################################################### end new std out tab 
         
        
        # Add tabs to tab widget 
        self.tabs.addTab(self.h1Tab,"ERP")
        self.tabs.addTab(self.neuroTab,"Neuropsych")
        self.tabs.addTab(self.redirectTab, "Stdout & Stderr")

        
        self.setCentralWidget(self.tabs)
        self.show()
        
        
        sys.stdout = EmittingStream(textWritten=self.redirect_output)
        
        
    def redirect_output(self, text):

        """Append text to the QTextEdit."""

        # Maybe QTextEdit.append() works as well, but this is how I do it:

        cursor = self.process.textCursor()
        cursor.movePosition(QTextCursor.End)
        cursor.insertText(text)
        self.process.setTextCursor(cursor)
        self.process.ensureCursorVisible()



    def __del__(self):
        sys.stdout = sys.__stdout__
    ###################################################################
        
    def clear_function(self, signal):
        self.process.clear()
        
    def test(self, signal):#### remove this function ######
        
        directoryInp = self.dirInp.text().strip()
        self.erp_and_filesize_check(directoryInp)
        

        
        
    ############################################## ERP HANDLERS ##############################################
    
    def h1_handler(self, signal):
        """ add a directories to exclude box?? """
        
        directoryInp = self.dirInp.text().strip()
        directorytrg = self.trgInp.text().strip()
        
        
        files = self.filesInp.text().strip()
        files_lst = files.split(' ')
        files_set = set(files_lst)
        
        dirs = [os.path.join(r,n) for r,d,f in os.walk(p1) for n in d]

        count = 0
        for i in dirs:
            count+=1
            print("\n\n{}".format(count)), sd.get_h1s(i, files_set, del_ext=True, trg_dir=directorytrg)
            
            
    def review_handler(self, signal):
        directoryInp = self.dirInp.text().strip()
        
        dirs = [os.path.join(r,n) for r,d,f in os.walk(directoryInp) for n in d]
        
        count = 0
        for i in dirs:
            count+=1
            print("\n\n{}".format(count)), ep.run_all(i)
        
            
            
    def erp_shell_scripts(self, signal):
        
        buttonReply = QMessageBox.question(self, 'Confirmation Message', "Are you sure?", QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        if buttonReply == QMessageBox.Yes:
            directoryInp = self.dirInp.text().strip()
            ep.iter_shell_check(directoryInp)
        else:
            print('No clicked.')
        
        
    ############################################## NEURO HANDLERS ##############################################
    
    def check_neuro_handler(self, signal):
        """Check to see if new neuropsych data already exists in /raw_data/neuropsych/__sitename__"""
        
        directoryInp = self.dirInpNeuro.text().strip()
        siteInp = self.siteInpNeuro.text().strip()
        # my script
        np_run_exists(directoryInp, siteInp)
        
        
    def dupes_neuro_handler(self, signal):
        """Check for duplicate files in new neuropsych data from site"""
        
        directoryInp = self.dirInpNeuro.text().strip()
        # my script 
        md5_check_walk(directoryInp)
        
        
    def review_neuro_handler(self, signal):
        """Check for file naming/xml inconsistencies in new neuropsych data from site"""
        
        directoryInp = self.dirInpNeuro.text().strip()
        monthInp = self.monthInpNeuro.text().strip()
        # my script 
        neuro(directoryInp, int(monthInp))
        

    def move_neuro_handler(self, signal):
        """only after the above 3 functions are run, move new site data to /raw_data/neuropsych/__sitename__"""
        
        directoryInp = self.dirInpNeuro.text().strip()
        siteInp = self.siteInpNeuro.text().strip()
        # my script
        move_neuro_files(directoryInp, siteInp)
        
    ############################################## END NEURO HANDLERS ##############################################
        
if __name__ == '__main__':
    app = QCoreApplication.instance() ### adding this if statement prevents kernel from crashing 
    if app is None:
        app = QApplication(sys.argv)
        print(app)
    ex = App()
    sys.exit(app.exec_())

## 1st neuro button  -- works

In [2]:
def np_run_exists(path_to_new_data, site):
    '''given a path to new data and str(site) -- 
       tells if file already exists in directory'''
    
    np_basename = '/vol01/raw_data/neuropsych/'
    
    files = []
    for r,d,f in os.walk(path_to_new_data):
        for n in f:
            files.append(n)
            
         
    np_full_path = np_basename + site
    
    duplicate_files = []  
    for r,d,f in os.walk(np_full_path):
        for n in f:
            if n in files:
                path = os.path.join(r,n)
                duplicate_files.append(path)
    
    if len(duplicate_files) == 0:
        print('All files are unique')
    else:
        #pass
        #print('im here')
        #print('The following files already exist in', np_full_path, ' --->\n\n', [i for i in duplicate_files])
        print('The following files already exist in {}...\n'.format(np_full_path))
        count = 0
        for idx, i in enumerate(duplicate_files):
            if idx % 5 == 0:
                print('\n{}'.format(i))
            else:
                print(i)

## 2nd neuro button -- works

In [ ]:
#!/usr/bin/env python
import hashlib
from collections import defaultdict
import os 

def md5(path):
    '''generate md5 - -read file in binary mode'''
    with open(path, "rb") as f:
        data = f.read()
        md5_return = hashlib.md5(data).hexdigest()
        return md5_return
    
    
    
def md5_check_walk(dir_path):
    '''given a dir path -- comapres md5 checksums across files and identifies duplicate files '''
    md5_dict = defaultdict(list)
    for r,d,f in os.walk(dir_path):
        for n in f:
            fp = os.path.join(r,n)
            md5_dict[md5(fp)].append(fp)
            
    for k,v in md5_dict.items():
        if len(v) > 1:  #multiple values for the same key
            print(len(v), 'Duplicate files:', v,'\n\n\n', 'with checksum', '\n', k)
        elif len(v) == 1:
            print('No duplicates found {}'.format(k))

## 3rd neuro button -- needs work

In [23]:
#!/usr/bin/env python
import os
import re
import sys
from collections import Counter
from collections import defaultdict
import datetime

def restart_script():
    input('Please fix then press enter to restart program')
    neuro(fp, 10)
    
def check_dict_len(dictionary, len_to_check):
    '''used to check whether a file is empty or not '''
    
    for k,v in dictionary.items():
        if len(v) < len_to_check:
            print('Error:', k, 'is likely an empty file')
    
def neuro(fp, month_as_int):
    
    #1 -- check for unknown file types
    for r,d,f in os.walk(fp):
        for n in f:
            if not n.endswith(("xml", "txt", "_sum.txt")):
                path  = os.path.join(r,n)
                print('\n', 'Error: Unknown file types found --', path)
                #sys.exit(1)


    #2 -- check that all IDS have 8 numbers and all sum/txt files having CBST/TOLT in filename
    for r,d, f in os.walk(fp):
        for n in f:
            if n.endswith(("_sum.txt", 'txt')):
                split = n.split("_")
                if len(split[0]) != 8:
                    print('Error: ID Incorrect Length -- ', split)
                    sys.exit(1)
                else:
                    if split[1] == 'CBST':
                        pass
                    elif split[1] == 'TOLT':
                        pass
                    else:
                        print('Error: Wrong experiment name -- ', split[0])
                        #sys.exit(1)
                        
    # 3 -- check for empty files                     
    xml_size_dict = {}
    txt_size_dict = {}
    for r,d,f in os.walk(fp):
        for n in f:
            if n.endswith('xml'):
                with open(os.path.join(r,n), 'r') as file:
                    for xml_line in file:
                        path_xml= os.path.join(r,n)
                        xml_size_dict.setdefault(path_xml, []).append(xml_line)
            if n.endswith(('txt', '_sum.txt')):
                with open(os.path.join(r,n), 'r') as file_txt:
                    for txt_line in file_txt:
                        path_txt= os.path.join(r,n)
                        txt_size_dict.setdefault(path_txt, []).append(txt_line)

    check_dict_len(xml_size_dict, 3)

    check_dict_len(txt_size_dict, 3)

    #4 -- create dictionary of each element in the filename 
    sum_txt_dict = {}
    txt_dict = {}
    xml_fname_dict = {}

    for r,d,f in os.walk(fp):
        ids_sumtxt = []
        version_sumtxt = []
        run_sumtxt = []
        ext_sumtxt = []

        ids_txt=[]
        version_txt = []
        run_txt = []
        ext_txt = []


        ids_xml = []
        run_xml = []
        ext_xml = []
        for n in f:
            if n.endswith('txt'):
                if n.endswith('_sum.txt'):
                    sumtxt_split = n.split('_')
                    ids_sumtxt.append(sumtxt_split[0])
                    version_sumtxt.append(sumtxt_split[2])
                    run_sumtxt.append(sumtxt_split[3])
                    ext_sumtxt.append(sumtxt_split[-1])
                else:
                    txt_split = re.split(r'[_.]', n)
                    ids_txt.append(txt_split[0])
                    version_txt.append(txt_split[2])
                    run_txt.append(txt_split[3])
                    ext_txt.append(txt_split[-1])
            if n.endswith('xml'):
                xml_split = n.split('_')
                ids_xml.append(xml_split[0])
                run_xml.append(xml_split[1])
                ext_xml.append(xml_split[2])

        tup_sumtxt = version_sumtxt, run_sumtxt, ext_sumtxt
        tup_txt = version_txt, run_txt, ext_txt
        tup_xml = run_xml, ext_xml

        for i in ids_sumtxt:
            sum_txt_dict[i] = tup_sumtxt

        for j in ids_txt:
            txt_dict[j] = tup_txt

        for k in ids_xml:
            xml_fname_dict[i] = tup_xml 

        ids_total = ids_sumtxt + ids_txt + ids_xml
        ids_count_dict = Counter(ids_total)
        for k,v in ids_count_dict.items():
            if v != 5:
                print('Error: There\'s an ID missing from', k)
                #sys.exit(1)

    #####might need to add check to make sure sum_txt & txt dict keys are same length???

    #5 -- check for version, run leter, and # of extensions for sum_txt files 
    for k,v in sum_txt_dict.items():
        if len(list(set(v[0]))) != 1:
            print('Error: Check version number for sum_txt file', k)
        for i in v[0]:
            if i != '1':
                print('Error: Check version number for sum_txt file', k)
        if len(list(set(v[1]))) != 1:
            print('Error: Check run letter for sum_txt file', k)
            sys.exit(1)
        if len(v[2]) != 2:
            print('Error: Missing sum_txt file for ID', k)


    #6 -- check for version, run leter, and # of extensions for txt files 
    for k,v in txt_dict.items():
        if len(list(set(v[0]))) != 1:
            print('Error: Check version number for txt file', k)
        for i in v[0]:
            if i != '1':
                print('Error: Check version number for txt file', k)
        if len(list(set(v[1]))) != 1:
            print('Error: Check run letter for txt file', k)
            sys.exit(1)
        if len(v[2]) != 2:
            print('Error: Missing txt file for ID txt file', k)


    #7 -- check to see if version and run letter are identical across txt & sum_txt files
    for k,v in sum_txt_dict.items():
        if k in txt_dict.keys():
            if not v[0] == txt_dict[k][0]:
                print('Error: Version number between sum.txt & txt files don\'t match for', k)
            if not v[1]  == txt_dict[k][1]:
                print('Error: Run letter between sum.txt & txt files don\'t match for', k)
                sys.exit(1)

    #8 -- check for things inside xml file            
    xml = [os.path.join(root,name) for root,dirs,files in os.walk(fp) for name in files if name.endswith(".xml")]

    within_xml_id = []
    within_xml_run = []
    xml_motiv = []


    xml_dict_new = {}
    for i in xml:
        lst_xml_id = []
        dob_lst = []
        testdate_lst = []
        gender_lst = []
        hand_lst = []
        with open(i) as f:
            for line in f:
                if line.startswith('  <Sub'):
                    lst_xml_id.extend(re.findall(r'<SubjectID>(.*?)</SubjectID>', line))
                    within_xml_id.extend(re.findall(r'<SubjectID>(.*?)</SubjectID>', line))
                if line.startswith('  <Sess'):
                    within_xml_run.extend(re.findall(r'<SessionCode>(.*?)</SessionCode>', line))
                if line.startswith('  <Motivation>'):
                    xml_motiv.extend(re.findall(r'<Motivation>(.*?)</Motivation>', line))
                if line.startswith('  <DOB>'):
                    dob_lst.extend(re.findall(r'<DOB>(.*?)</DOB>', line))
                if line.startswith('  <TestDate>'):
                    testdate_lst.extend(re.findall(r'<TestDate>(.*?)</TestDate>', line))
                if line.startswith('  <Gender>'):
                    gender_lst.extend(re.findall(r'<Gender>(.*?)</Gender>', line))
                if line.startswith('  <Hand>'):
                    hand_lst.extend(re.findall(r'<Hand>(.*?)</Hand>', line))
                    
                xml_tuples = dob_lst, testdate_lst, gender_lst, hand_lst
                for i in lst_xml_id:
                    xml_dict_new[i] = xml_tuples
                    
    now = datetime.datetime.now().year
    

    #9 -- xml check for accurate DOB, testdate, & that gender and version start with uppercase letter
    for k,v in xml_dict_new.items():
        for i in v[0]:
            date_split = i.split('/')
            year = int(date_split[-1])
            if year > 2015:
                print('Error: Check DOB in xml file for', k)
        for j in v[1]:
            testdate_split = j.split('/')
            year_test = int(testdate_split[-1])
            if year_test != now:
                print('Error: Check test date in xml file for', k)
            if int(month_as_int) != int(testdate_split[0]):
                print('Error: Month of test date in xml file incorrect for', k )
        for l in v[2]:
            if not l[0].isupper(): #maybe str.istitle() would be better?
                print('Error: Make gender uppercase in xml file for', k)
        for m in v[3]:
            if not m[0].isupper():
                print('Error: Make handedness uppercase in xml file for', k)

    inside_xml = dict(zip(within_xml_id, within_xml_run))

    #10 -- xml check for length of sub IDs 
    xml_ids = [('Error: Check inside xml file for sub ID', i) for i in within_xml_id if len(i) != 8]
    if len(xml_ids) != 0:
        print(xml_ids)

    #11 -- 12 will only work if 2 xml dicts have the same keys 
    missing_xml_keys = set(xml_fname_dict.keys() ^ inside_xml.keys())
    
    if len(list(missing_xml_keys)) > 0:
        print('Error: Check', missing_xml_keys, 'folder for missing xml files/'
              'incorrect sub ID within xml file' )
        sys.exit(1)

    #12 -- check to see if run letter within xml matches xml filename    
    for k,v in xml_fname_dict.items():
        for l in v[0]:
            if k in inside_xml.keys():
                if l not in inside_xml[k]:
                    print('Error: Check', k, 'folder for mismatch between xml run letter and xml filename run letter')
                    sys.exit(1)

    #13 -- since we know txt & sum_txt run letters are identical & that inside and xml fname run letters are identical...
    #13... check run letters against xml & sum_txt files 
    for k,v in txt_dict.items():
        run_letter = v[1][0][0]
        if k in inside_xml.keys():
            if run_letter not in inside_xml.values():
                print('Error: Check', k, 'folder for mismatch between xml run letter and xml filename run letter')
                sys.exit(1)

## Review site data  -- 1st & 3rd buttons -- both work

In [ ]:
import os
from glob import glob
from collections import defaultdict
import re
import subprocess

class erp_data:

    def __init__(self):

        # avg + ps exp names are the same
        self.avg_and_ps_exps = ['vp3', 'cpt', 'ern', 'ant', 'aod', 'anr', 'stp', 'gng']
        # number of files associated with avg extension 
        self.avg_exp_nums = [3,6,4,4,2,2,2,2]
        # cnt exp names
        self.cnt_exp_list = ['eeo', 'eec', 'vp3', 'cpt', 'ern', 'ant', 'aod', 'ans', 'stp', 'gng']
        # versions associated with erp 
        self.version_list = ['4', '4', '6', '4', '9', '6', '7', '5', '3', '3']
        # dat exp names 
        self.dat_exps = ['vp3', 'cpt', 'ern', 'ant', 'aod', 'ans', 'stp', 'gng']
        # only 1 exp for dat/cnt/ps
        self.exp_nums_single = [1] * len(self.cnt_exp_list)


    def check_erp_version(self, path, exp_name, version_num): 
        """
        returns whether ERP experiment version is correct 
        """
        self.path = path
        self.exp_name = exp_name
        self.version_num = version_num

        for r,d,f in os.walk(path):
            for n in f:
                split = n.split('_')
                if n.startswith((exp_name)):
                    if split[1] != version_num:
                        return 'Check version for {}'.format(n)


    def iter_check_version(self, path, cnt_exp_list, version_list):
        """
        iterator version of check_erp_version()
        """
        self.path = path


        for exp, version in zip(self.cnt_exp_list, self.version_list):
            out = self.check_erp_version(path, exp, version)
            if out is None:
                pass
            else:
                return str(out)


    def parse_site_data(self, path):
        """
        returns a nested dictionary of common/uncommon file extensions by experiment for a sub
        """
        self.path = path

        key = path.split('/')[-1]

        nested_dict = {}
        for r,d,f in os.walk(path):
            for n in f:
                if n.endswith('_32.cnt'):
                    cnts = n.split('_')
                    nested_dict.setdefault(key, {}).setdefault('cnt', []).append(cnts[0])
                if n.endswith('dat'):
                    dats = n.split('_')
                    nested_dict.setdefault(key, {}).setdefault('dat', []).append(dats[0])
                if n.endswith('_avg.ps'):
                    pss = n.split('_')
                    nested_dict.setdefault(key, {}).setdefault('ps', []).append(pss[0])
                if n.endswith('.avg'):
                    avgs = re.split(r'[_.]', n)
                    nested_dict.setdefault(key, {}).setdefault('avg', []).append(avgs[0])
                if n.endswith('_orig.cnt'):
                    origs = n.split('_')
                    nested_dict.setdefault(key, {}).setdefault('orig_cnt', []).append(origs[0])
                if n.endswith('_32_original.cnt'):
                    bad_orig = n.split('_')
                    nested_dict.setdefault(key, {}).setdefault('bad_orig', []).append(bad_orig[0])
                if n.endswith('_rr.cnt'):
                    reruns = n.split('_')
                    nested_dict.setdefault(key, {}).setdefault('rerun', []).append(reruns[0])
                if n.endswith('_cnt.h1'):
                    cnt_h1 = n.split('_')
                    nested_dict.setdefault(key, {}).setdefault('cnt_h1', []).append(cnt_h1[0])
                if n.endswith('_avg.h1'):
                    avg_h1 = n.split('_')
                    nested_dict.setdefault(key, {}).setdefault('h1', []).append(avg_h1[0])
                if n.endswith('_avg.h1.ps'):
                    h1_ps = n.split('_')
                    nested_dict.setdefault(key, {}).setdefault('h1_ps', []).append(h1_ps[0])

        return nested_dict


    def remove_wild_files(self, path):
        """
        prompts user to delete file extensions that don't belong in ns folders
        """
        self.path = path

        wild_files = []
        for r,d,f in os.walk(path):
            for n in f:
                if n.endswith(('_32.cnt', '_orig.cnt', 'avg', 'avg.ps', 'dat', 'txt', 'sub')):
                    pass
                else:
                    fp = os.path.join(r,n)
                    wild_files.append(fp)
        count = 0
        for wild in wild_files:
            count+=1
            print("\n{} || {}".format(count, wild))
            ans= input('\nDo you want to delete this file?\n> ')
            if ans in ['y', 'Y', 'Yes', 'yes']:
                #os.remove(wild)
                print('deleting')
            else:
                pass


    def get_ext_count(self, path, nested_dict, ext_type, exp_name, number_files):
        """
        checks nested dictionary for number of extensions associated with each experiment
        e.g. checks that there's 6 CPT avg files or 1 EEC cnt file...
        """

        self.path = path
        self.nested_dict = nested_dict
        self.ext_type = ext_type
        self.exp_name = exp_name
        self.number_files = number_files

        output = []
        for k,v in nested_dict.items():
            for k1, v1 in v.items():
                if ext_type == k1:
                    num_files = v1.count(exp_name)
                    if num_files != number_files:
                        return 'Incorrect number of {} {} files in {}'.format(exp_name, ext_type, path)


    # get_ext_count()
    def iter_exps(self, path, nested_dict, exp_list, exp_list_avgs, ext_type):
        """
        iterator version of iter_exps()
        """
        self.path = path
        self.exp_list = exp_list
        self.exp_list_avgs = exp_list_avgs
        self.ext_type = ext_type

        for exp, avg_nums in zip(exp_list, exp_list_avgs):
            out = self.get_ext_count(path, nested_dict, ext_type, exp, avg_nums)
            if out is None:
                pass
            else:
                return str(out)


    def check_id_and_run(self, path):
        """
        checks to see if important file extensions have same sub ID & run letter
        """
        self.path = path

        folder = path.split('/')[-1]

        sub_id_list = []
        run_letter_list = []

        for file in glob(os.path.join(path, '*.*')):
            if file.endswith(('_32.cnt', '_orig.cnt', '_avg.ps', '.avg', 'dat')):
                fname = os.path.basename(file)
                if not fname.endswith(('avg', 'dat')):
                    sub_id = fname.split('_')[3]
                    sub_id_list.append(sub_id)
                else:
                    avg_dat_ids = re.split(r'[_.A-Z]', fname)
                    sub_id_list.append(avg_dat_ids[3])

                # append run letters 
                run_letter_list.append(fname.split('_')[2][0])


        unique_ids = list(set(sub_id_list))
        unique_run_letters = list(set(run_letter_list))

        if  len(unique_ids) > 1:
            return str("Folder {} has more than one sub ID => {}".format(folder, unique_ids))

        if len(unique_run_letters) > 1:
            return str("Folder {} has more than one run letter => {}".format(folder, unique_run_letters))


    def print_erp_version(self, path, cnt_exp_list, version_list):
        """
        check erp version 
        """

        self.path = path

        print('\n\nERP VERSION CHECK:')

        iter_check = self.iter_check_version(path, self.cnt_exp_list, self.version_list)
        if  iter_check is None:
            print('All versions check out!')
        else:
            print(iter_check)


    def print_file_counts(self, nested_data_dict):
        """
        returns counts of file extensions
        """

        self.nested_data_dict = nested_data_dict

        print('\n\nFILES COUNT:')
        for k,v in nested_data_dict.items():
            for k1,v1 in v.items():
                print("There are {} {} files".format(len(v1), k1.upper()))


    def print_missing_exps(self, path, nested_data_dict):
        """
        returns file type that's missing, if any 
        """
        self.path = path
        self.nested_data_dict = nested_data_dict

        print('\n\nMissing Experiments:')


        avg_counts = self.iter_exps(path, nested_data_dict, self.avg_and_ps_exps, self.avg_exp_nums, 'avg')

        if avg_counts is None:
            print('All avg files found!')
        else:
            print(avg_counts)


        cnt_counts = self.iter_exps(path, nested_data_dict, self.cnt_exp_list, self.exp_nums_single, 'cnt')
        if cnt_counts is None:
            print('All cnt files found!')
        else:
            print(cnt_counts)


        ps_counts = self.iter_exps(path, nested_data_dict, self.avg_and_ps_exps, self.exp_nums_single, 'ps')
        if ps_counts is None:
            print('All ps files found!')
        else:
            print(ps_counts)


        dat_counts = self.iter_exps(path, nested_data_dict, self.dat_exps, self.exp_nums_single, 'dat')
        if dat_counts is None:
            print('All dat files found!')
        else:
            print(dat_counts)


    def print_wild_files(self, path):
        """
        returns files that don't belong
        """
        self.path = path

        print("\n\nFILES THAT DON'T BELONG IN NS FOLDERS:")
        self.remove_wild_files(path)


    def print_id_and_letter(self, path):
        """
        returns ID & run letter, should both be unique
        """
        self.path = path

        print('\n\nCHECK SUBJECT ID & RUN LETTER:')

        if self.check_id_and_run(path) is None:
            print('All IDs & run letters check out!')
        else:
            print(self.check_id_and_run(path))


    def run_all(self, path):
        """
        2nd to last step 
        """
        self.path = path

        nested_data_dict = self.parse_site_data(path)

        self.print_erp_version(path, self.cnt_exp_list, self.version_list)
        self.print_file_counts(nested_data_dict)
        self.print_missing_exps(path, nested_data_dict)
        self.print_id_and_letter(path)


    def execute_all(self, path):
        """
        last step
        """

        self.path = path

        test = [os.path.join(r,n) for r,d,f in os.walk(path) for n in d]

        if len(test) == 0:
            self.run_all(path)
        else:
            fps = [os.path.join(r,n) for r,d,f in os.walk(path) for n in d]
            count=0
            for i in fps:
                print("\n\n{} || {}".format(count, i))
                self.run_all(i)
                count+=1
                

    # anything below here doesn't get executed in execute_all()
    def shell_filesize_check(self, path):
        """
        return stdout & stderr from David's ERP shell scripts 
        """
        self.path = path

        erp_check = subprocess.Popen('ERP-raw-data_check.sh {}'.format(path), shell=True, 
                                      stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=path)
        result_erp = erp_check.communicate()[0]
        print("ERP CHECK: {} {}".format(path, result_erp.decode('ascii')))
        
        
        size_check = subprocess.Popen('DVD-file-size_check.sh {}'.format(path), shell=True, 
                                      stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=path)
        result_size = size_check.communicate()[0]
        print('FILE SIZE CHECK: {} {}\n\n'.format(path, result_size.decode('ascii')))


    def iter_shell_check(self, path):
        """
        iterate over directories checking with shell scripts 
        """

        self.path = path

        test = [os.path.join(r,n) for r,d,f in os.walk(path) for n in d]

        if len(test) == 0:
            self.shell_filesize_check(path)
        else:
            fps = [os.path.join(r,n) for r,d,f in os.walk(path) for n in d]
            count=0
            for i in fps:
                print("\n{} || {}".format(count, i))
                self.shell_filesize_check(i)
                count+=1


ep = erp_data()

## Create h1 button -- 2nd button -- works

In [ ]:
import os
import sys
from collections import defaultdict
import shutil
import subprocess

class site_data:
    #get_h1s()
    def __init__(self):
    
        self.ant_set = {'ant'}
        self.ans_set = {'ans'}
        self.other_exps = {'vp3', 'cpt', 'ern', 'aod', 'stp', 'gng'}
                
    def check_cnt_copy(self, path, exp_tuple):
        """
        checks to see if files you want to h1 are actually in the directory to begin with
        """

        self.path = path
        self.exp_tuple = exp_tuple

        cnt_dict = {}
        for r,d,f in os.walk(path):
            for n in f:
                if n.startswith((exp_tuple)) and n.endswith('_32.cnt'):
                    split = n.split('_')
                    cnt_dict.setdefault(split[3], []).append(split[0])


        for k,v in cnt_dict.items():
            if len(v) is not len(exp_tuple):
                missing_exp = '.'.join(str(s) for s in (set(v) ^ set(list(exp_tuple))))
                print('\n\nLOOK HERE!!!{} cnt file missing from {}\n\n'.format(missing_exp.upper(), path))
                
    #get_h1s()                
    def rename_cnts(self, path, skip=False):
        """
        rename file ending with _rr.cnt if it doesn't already exist
        """
        self.path = path
        self.skip = skip

        for r,d,f in os.walk(path):
            for n in f:
                if n.endswith('_rr.cnt'):
                    new_name = os.path.join(r, n[:-7] + '.cnt')
                    if not os.path.exists(new_name):
                        path = os.path.join(r,n)
                        os.rename(os.path.join(r,n), new_name)
                        print("Renaming {}".format(n))
                    else:
                        print("{} already exists!!".format(new_name))
        
        if skip:
            for r,d,f in os.walk(path):
                for n in f:
                    if n.endswith('_rr.cnt'):
                        print("Re-run found for {} -- create manually".format(n))
    #get_h1s()                    
    def create_cnth1(self, path):
        """
        create cnt.h1 files from shell script
        """
        self.path = path
        
        print('\n\n>>> MAKING CNT.H1 FILES <<<\n') 
        for r,d,f in os.walk(path):
            for n in f:
                if n.startswith(self.cnth1_tups) and n.endswith('_32.cnt'):
                    path = os.path.join(r,n)
                    p = subprocess.Popen("create_cnthdf1_from_cntneuroX.sh {}".format(path),shell=True, 
                                          stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(path))
                    result = p.communicate()[1]
                    print(result.decode('ascii'))

    #get_h1s()
    def create_avgh1(self, path):
        """
        create avg.h1 files from shell script
        """
        self.path = path

        print('\n\n>>> Making AVG.H1 FILES <<<\n') 
        for r,d,f in os.walk(path):
            for n in f:
                if n.startswith(self.ant_tup) and n.endswith('cnt.h1'):
                    ant_path = os.path.join(r,n)
                    p_ant = subprocess.Popen('create_avghdf1_from_cnthdf1X -lpfilter 8 -hpfilter 0.03 -thresh 75 -baseline_times -125 0 {}'.format(ant_path), 
                                             shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(ant_path))
                    result_ant = p_ant.communicate()[1]
                    print(result_ant.decode('ascii'))
                if n.startswith(self.ans_tup) and n.endswith('cnt.h1'):
                    ans_path = os.path.join(r,n)
                    p_ans = subprocess.Popen('create_avghdf1_from_cnthdf1X -lpfilter 16 -hpfilter 0.03 -thresh 100 -baseline_times -125 0 {}'.format(ans_path), 
                                            shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(ans_path))
                    result_ans = p_ans.communicate()[1]
                    print(result_ans.decode('ascii'))
                if n.startswith((self.others_tup)) and n.endswith('cnt.h1'):
                    path=os.path.join(r,n)
                    p_others = subprocess.Popen('create_avghdf1_from_cnthdf1X -lpfilter 16 -hpfilter 0.03 -thresh 75 -baseline_times -125 0 {}'.format(path), 
                                                shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(path))
                    result_others = p_others.communicate()[1]
                    print(result_others.decode('ascii'))
    #get_h1s()
    def create_avgps(self, path):
        """
        create avg.h1.ps files from shell script
        """
        
        self.path = path

        print('\n\n>>> Making AVG.PS FILES <<<\n') 
        for path, subdirs, files in os.walk(path):
            for name in files:
                if name.endswith("avg.h1"):
                    ps_paths = os.path.join(path, name)
                    subprocess.Popen("plot_hdf1_data.sh {}".format(name), shell=True, cwd=os.path.dirname(ps_paths))
                    print("creating ps files.. " + name)
                    
        #get_h1s()                
    def delete_bad_files(self, path, exts_to_keep=None, to_be_deleted_set=None):
        ''' returns any extension not in exts_to_keep & prompts user to delete '''
    
        self.path = path
        self.exts_to_keep = exts_to_keep
        self.to_be_deleted_set = to_be_deleted_set


        
        print("\n\n>>> REMOVING FILES <<<\n") 
        if exts_to_keep:

            for r,d,f in os.walk(path):
                for n in f:
                    if n.endswith(('_cnt.h1')):
                        os.remove(os.path.join(r,n))
                        print('Removing {}'.format(n))

        if to_be_deleted_set:
            
            for r,d,f in os.walk(path):
                for n in f:
                    if n.endswith(('_32.cnt', '_cnt.h1')):
                        os.remove(os.path.join(r,n))
                        print('Removing {}'.format(n))



    
    def get_h1s(self, path, set_of_exps, del_ext=None, ps=None, trg_dir=None):
        '''combines all these commands together'''
        
        self.path = path
        self.set_of_exps = set_of_exps
        self.ps = ps
        self.trg_dir = trg_dir
        
        #use in create_cnth1()
        self.cnth1_tups = tuple(set_of_exps)
        
        if_ant = self.ant_set & set_of_exps
        if_ans = self.ans_set & set_of_exps
        all_others = self.other_exps & set_of_exps
        
        #used in create_avgh1()
        self.ant_tup = tuple(if_ant)
        self.ans_tup = tuple(if_ans)
        self.others_tup = tuple(all_others)

        #if being used for peak picking, create new directories and move all cnt files to the correct folder...and so on
        if trg_dir:
            #create new directories
            exps_list = list(set_of_exps)
            for exp in exps_list:
                new_dirs = os.path.join(trg_dir, exp)
                if not os.path.exists(new_dirs):
                    os.makedirs(new_dirs)
                    print(">>> Creating {} <<<".format(new_dirs))
                else:
                    #print("{} already exist".format(new_dirs))
                    pass
            #copy cnt files to newly created directories       
            count = 0
            print('\n>>> COPYING CNT FILES <<<\n')
            for r,d,f in os.walk(path):
                for n in f:
                    if n.startswith(self.cnth1_tups) and n.endswith('_32.cnt'):
                        count+=1
                        print('Copying {}'.format(n))
                        shutil.copy(os.path.join(r,n), os.path.join(trg_dir, n[:3]))
            print('\nCopied {} of {} files'.format(count, len(self.cnth1_tups)))

            self.rename_cnts(path)
            self.check_cnt_copy(trg_dir, self.cnth1_tups)
            self.create_cnth1(trg_dir)
            self.create_avgh1(trg_dir)
            if ps:
                self.create_avgps(trg_dir)
            if del_ext:
                self.delete_bad_files(trg_dir, to_be_deleted_set=True)
            return True
        
        #if you want to create avg.h1s IN directory copying files...
        self.rename_cnts(path, skip=True)
        self.create_cnth1(path)
        self.create_avgh1(path)
        if ps:
            self.create_avgps(path)
        if del_ext:
            self.delete_bad_files(path, exts_to_keep=True)


sd = site_data()